# Import all datasets

In [1]:
import pandas as pd
import numpy as np

In [4]:
from import_budget_data import df_budget
from import_wb_api import df_gdp_monthly, df_inflation_rotated_monthly, df_interest_rotated_monthly
from import_bcb_api import df_macro_monthly
from import_bcb_currencies import df_currencies_monthly
from import_bcb_focus import df_focus_indicators_monthly
from import_imf_api import df_commodities
from import_ipea_api import df_revenue

c:\Users\augus\Meu Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\scr\import_budget_data.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_budget_raw[value] = df_budget_raw[value].str.replace('.', '').str.replace(',', '.').astype(float)
c:\Users\augus\Meu Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\scr\import_budget_data.py:70: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_step= df_function.groupby(['year', 'code']).sum().reset_index()
c:\Users\augus\Meu Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\scr\import_budget_data.py:88: FutureWarning: The default value of numeric_only in DataF


EntitySet (Endpoint): ExpectativasMercadoAnuais
EntityType: br.gov.bcb.olinda.servico.Expectativas.ExpectativaMercadoAnual
Properties: Indicador<str>, IndicadorDetalhe<str>, Data<str>, DataReferencia<str>, Media<float>, Mediana<float>, DesvioPadrao<float>, Minimo<float>, Maximo<float>, numeroRespondentes<int>, baseCalculo<int>


In [6]:
dataframes = [df_macro_monthly, # DF with brazilian macro data
              df_gdp_monthly, # DF with GDP data from other countries
              df_inflation_rotated_monthly, # DF with inflation data from other countries
              df_interest_rotated_monthly, # DF with interest rate data from other countries
              df_commodities, # DF with commodities historical prices
              df_budget, # DF with brazilian budget data
              df_focus_indicators_monthly, # DF with market expectations indicators from Brazilian Central Bank
              df_currencies_monthly,# DF with currencies historical prices - USD, EUR and CHN
              df_revenue] # DF with brazilian total revenue for federal government 

# Create a new DF with all dataframes merged
merged_df = df_macro_monthly # the merged df will start with the dataframe which contans the target variable

#loop to merge dataframes
for i,df in enumerate(dataframes):
    df.rename_axis('date', inplace=True) #rename index
    if df is not df_macro_monthly:
        merged_df = pd.merge(merged_df, df, on='date', how='left') #merge dataframes


In [7]:
'''
Some data preparation
'''

# Coverting all columns to float
cleanned_df = merged_df.replace('', np.nan).copy() #replace empty cells with NaN
cleanned_df = cleanned_df.astype(float)

# Fill NaN values with the next value of the column
#merged_df['eco_interest_rate'].fillna(method='bfill')

# Fill NaN values with the value before of the column
#merged_df['eco_interest_rate'].fillna(method='ffill')

# Fill NaN values interpollating the values of the column
cleanned_df.interpolate(limit_area='inside', inplace = True)

# Fill NaN values in the beggining of each time series with the next value of the column
cleanned_df.bfill(limit=None, inplace = True)




In [8]:
# Drop some columns, due to lack of data
cleanned_df.drop(columns=['exp_gross_public_debt_y+1', 'exp_gross_public_debt_y', 'exp_gross_public_debt_y+2', 'ARG_inflation', 'eco_gross_debt_gdp_pos', 'eco_gross_debt_R$_pos'], inplace=True)


In [9]:

# Check for NaN values
#checks_df = merged_df.info()
nan_counts = cleanned_df.isna().sum()
print(nan_counts.sort_values(ascending=False)) #print NaN counts for each column

nan_counts.sort_values(ascending=False).to_csv('nan_counts.txt', header=False)





US_GDP                             6
UK_GDP                             6
eco_fiscal_result_month            3
eco_net_debt_gdp_%_federal_govt    3
eco_fiscal_result_12months         3
                                  ..
exp_GDP (%)_y+1                    1
eco_interest_rate                  0
CNY                                0
EUR                                0
USD                                0
Length: 79, dtype: int64


In [10]:
# Export merged df to csv
print(len(cleanned_df))
print(len(cleanned_df.columns))
cleanned_df.to_csv('../data/cleanned_df.csv')

278
79
